In [ ]:
!pip install yfinance
!pip install requests

Gather Stock Data

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker, start_date, end_date):
    # Fetch stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    # Return only the "Close" prices
    return stock_data['Close']

# Example: Fetch data for Apple (AAPL) from January 1, 2010 to March 1, 2025
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2025-03-01'
stock_data = fetch_stock_data(ticker, start_date, end_date)

# Show only the Close prices
print(stock_data.head())


Gathering Articles

In [ ]:
import requests
import json
from datetime import datetime
import time

# Function to fetch articles from GNews API
def fetch_articles_for_year(api_key, company_name, year):
    # Set start and end dates for the year
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    
    # Construct the URL
    url = f'https://gnews.io/api/v4/search?q={company_name}&from={start_date}&to={end_date}&lang=en&max=10&token={api_key}'
    
    # Send the request to GNews API
    response = requests.get(url)
    
    if response.status_code == 200:
        articles = response.json()
        return articles.get('articles', [])
    else:
        print(f"Error fetching articles for {year}: {response.status_code}")
        return []

# Function to fetch articles for each year from 2010 to now
def fetch_articles(api_key, company_name):
    # Get the current year
    current_year = datetime.now().year
    
    all_articles = {}
    
    # Loop through each year from 2010 to current year
    for year in range(2010, current_year + 1):
        print(f"Fetching articles for {year}...")
        articles = fetch_articles_for_year(api_key, company_name, year)
        
        if articles:
            all_articles[year] = articles
        else:
            print(f"No articles found for {year}.")
        
        # Add a delay of 2 seconds between each request to avoid hitting the rate limit
        time.sleep(2)
    
    return all_articles

# Example usage
api_key = os.getenv("NEWS_API_KEY")
company_name = 'Tesla'  # Replace with the company you are interested in

# Fetch articles for each year
articles_data = fetch_articles(api_key, company_name)

# Print the results (first 5 articles from the first year)
for year, articles in articles_data.items():
    print(f"\nArticles from {year}:")
    for i, article in enumerate(articles[:5]):
        title = article['title']
        published_at = article['publishedAt']
        content = article['content']  # Extract content
        
        # Print the title, published date, and content
        print(f"{i+1}. {title} - {published_at}")
        print(f"Content: {content}\n")
